Introduction
============

This notebook will guide you through the mechanical basics of doing photometry. For these exercises we will use *photutils*, but there are many different other tools one can use to extract photometry from images (DAOPHOT, DOLPHOT, SEXTRACTOR, EPSF).
This tutorial is based on the [photutils](http://http://photutils.readthedocs.io) documentation. 

In [3]:
import numpy as np
from IPython.core.pylabtools import *
from pylab import *
from numpy import * 
from photutils import datasets
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pylab as plt

Image Display
-------------
We will now download and display an example image.

In [ ]:
hdu = datasets.load_star_image()    

data = hdu.data[0:400, 0:400]    

norm = ImageNormalize(stretch=LogStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm)
show()

Source Detection
----------------
Before we do photometry we want to find all the sources in the image. We first measure some statistics about the image. We are interested in the background statistics so we subtract it and also determine which stars are 5$\sigma$ detections. 

In [14]:
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder

mean, median, std = sigma_clipped_stats(data, sigma=3.0, iters=5)    
print(mean, median, std)  

3667.779240018601 3649.0 204.27923665845705


In [8]:
from photutils import datasets
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
mean, median, std = sigma_clipped_stats(data, sigma=3.0, iters=5) 
hdu = datasets.load_star_image()
data = hdu.data[0:400, 0:400]
from photutils import DAOStarFinder
sources = DAOStarFinder(fwhm=3.0, threshold=5.*std)    

from photutils import CircularAperture

positions = (sources)
apertures = CircularAperture(positions, r=5.)
apertures.plot(color='blue', lw=1.5, alpha=0.5)


TypeError: List or array of (x, y) pixel coordinates is expected, got "<photutils.detection.findstars.DAOStarFinder object at 0xb17267278>".

In [ ]:
from photutils import CircularAperture

positions = (sources['xcentroid'], sources['ycentroid'])
apertures = CircularAperture(positions, r=5.)
apertures.plot(color='blue', lw=1.5, alpha=0.5)

Aperture photometry
-------------------
Now that we have the positions of stars, we can proceed with measuring their fluxes. When we first measure the fluxes of stars, they include the sky background. We will subtract that later.

In [ ]:
from photutils import CircularAnnulus
from photutils import aperture_photometry

apertures_r3 = CircularAperture((sources['xcentroid'], sources['ycentroid']), r=3.)

phot_table = aperture_photometry(data, apertures_r3)
print(phot_table)

In the following cells we create annuli apertures to measure the average value of the pixels around each star. We multiply the average sky value by the number of pixels under the aperture and then subtract that value from the total star flux.

In [ ]:
annulus_apertures = CircularAnnulus(positions, r_in=9., r_out=12.)
rawflux_r3 = aperture_photometry(data, apertures_r3)
bkgflux_table = aperture_photometry(data, annulus_apertures)

bkg_mean = bkgflux_table['aperture_sum'] / annulus_apertures.area()
print(bkg_mean)

In [ ]:
bkg_sum = bkg_mean * apertures_r3.area()
print(bkg_sum)

In [ ]:
final_phot_r3 = rawflux_r3['aperture_sum'] - bkg_sum
print(final_phot_r3)

Aperture corrections
--------------------
The above photometry was performed with a small aperture to increase the signal to noise of our measurement. The problem with that is that we are missing some light from each star. To compensate we will do photometry with a large radius. The difference in magnitude from the two aperture measurements is the aperture correction. This correction is applied to all the stars at the end.

In [ ]:
apertures_r5 = CircularAperture((sources['xcentroid'], sources['ycentroid']), r=5.) #note the larger aperture
rawflux_r5 = aperture_photometry(data, apertures_r5)

bkg_sum = bkg_mean * apertures_r5.area()
final_phot_r5 = rawflux_r5['aperture_sum'] - bkg_sum

Now you need to convert these fluxes into magnitudes:

In [ ]:
mag_r3 = -2.5*np.log10(final_phot_r3)
mag_r5 = -2.5*np.log10(final_phot_r5)
deltamag = mag_r3 - mag_r5

In [ ]:
plt.clf()
plt.scatter(mag_r5,deltamag)

In [ ]:
plt.clf()
plt.scatter(mag_r5,mag_r3-mag_r5,c='k',edgecolors='none')
plt.axhline(ls='--',c='b')
plt.xlim(-15,-8)
plt.ylim(-1,1)
plt.xlabel('Mag [r=5]',fontsize=18)
plt.ylabel('$\Delta$mag',fontsize=18)

In [ ]:
mask = [(mag_r5>-13.)&(mag_r5<-11.)&(deltamag>0.)&(deltamag<0.4)]

mean, median, std = sigma_clipped_stats(deltamag[mask], sigma=3.0, iters=5)
apcor = median
print(apcor)

plt.axhline(apcor,ls='-',c='r')

In [ ]:
final_phot =-2.5*np.log10(final_phot_r3) + apcor + 25.

In [ ]:
print(final_phot)

Exercise
--------

Use the images you made in the Astrodrizzle exercise to make a color magnitude diagram (F606W-F814W vs F814W). 

1. You will first have to change the drizzled images back to counts. 
2. Find sources on one image, use the same catalog for both images.
3. Do photometry on each image, including aperture correction plots.
4. Match catalogs, and plot CMD.

Send me three plots: the aperture correction for each band and the CMD. Each plot should be clearly labeled. 

Below is some information you might need:

\begin{equation}
rescale = 0.03/0.049 \\
counts\_image = cps\_image \times exptime + \frac{\Sigma{mdrizsky}}{2}sky \times rescale^2
\end{equation}

The information you need can be found in the image header and in the fits table found in the last extension of the image.

In [ ]:
from astropy.io import fits

fits.info('F606W_drc.fits')
hdu = fits.open('F606W_drc.fits')
sci = hdu[1].data
hdr = hdu[0].header
tab = hdu[4].data
hdu.close()

In [ ]:
tab['mdrizsky']

In [ ]:
(0.03/0.05)**2*np.sum(tab['mdrizsky'])/2

In [ ]:
a = sci*hdr['texptime']+(0.03/0.05)**2*np.sum(tab['mdrizsky'])/2

In [ ]:
fits.writeto('F606W_cts.fits',a,header=hdr)

Tips
----

- Remember, on a CMD, the y-axis goes from bright stars at top to faint stars on the bottom.